<a href="https://colab.research.google.com/github/RaziehAkbari2020/Finetunning-Transformers/blob/main/Fine_tunning_Parsbert_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizer
import tensorflow as tf
import tensorflow_datasets as tfds

گام اول: خواندن دیتای آموزش

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/', force_remount=True)

data_path = '/content/train.csv'
df = pd.read_csv(data_path, sep='\t', encoding='utf-8')[:15000]
df = df.dropna()
df.head()

,Unnamed: 0,comment,label,label_id
0,0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1
1,1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0
2,2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1
3,3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0
4,4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0


In [ ]:
# ef preprocess_tweet(tweet_text):
#     # Remove usernames, hashtags, links, and extra characters
#     tweet_text = re.sub(r'(@\w+|http\S+|:)', '', tweet_text)
#     return tweet_text.strip()

In [ ]:
df['label'].value_counts()

label
HAPPY    7508
SAD      7492
Name: count, dtype: int64

گام دوم: جداسازی تست و ترین

In [ ]:
train_text, test_text, train_labels, test_labels = train_test_split(df['comment'], df['label_id'],
                                                                    random_state=50,
                                                                    test_size=0.2)


گام سوم: وان هات کردن در جایی که خصوصا لیبل ها زیاده

In [ ]:
one_hot = tf.keras.layers.CategoryEncoding(num_tokens=2, output_mode="one_hot")
train_labels = one_hot(train_labels)
test_labels = one_hot(test_labels)
test_labels

<tf.Tensor: shape=(3000, 2), dtype=float32, numpy=
array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]], dtype=float32)>

گام چهارم: شناسایی مدل مدنظر
 (دراینجا Pars Bert v3) انتخاب شد

In [ ]:
tokenizer = BertTokenizer.from_pretrained("HooshvareLab/bert-fa-zwnj-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/426k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

با توکنایزری که انتخاب کردیم اومده تومن کرده یه جمله رو

In [ ]:
tokenizer.tokenize(".سلام، ما می‌خواهیم مدل را به شیوه‌ای جدید اموزش بدهیم")

['.',
 'سلام',
 '،',
 'ما',
 'میخواهیم',
 'مدل',
 'را',
 'به',
 'شیوه',
 '##ای',
 'جدید',
 'اموزش',
 'بدهیم']

یا توکنایزر انتخاب شده اومده انکد کرده

In [ ]:
text = ["این یک مدل جالبه", "ما میخوایم مدل رو اموزش بدیم"]

# encode text
sent_id = tokenizer.batch_encode_plus(text, padding=True)

# output
sent_id

{'input_ids': [[2, 1930, 1961, 2496, 20757, 3, 0, 0], [2, 2121, 18339, 2496, 1957, 3054, 10112, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1]]}

گام پنجم: در واقع میاد تابع گرفتن کامنت و تبدیلش به انکد رو انجام میده


In [ ]:
def convert_example_to_feature(review, max_length=128): ### 128 چک شود
  return tokenizer.encode_plus(review,
                add_special_tokens = True, # add [CLS], [SEP]
                max_length = max_length,
                pad_to_max_length = True,
                truncation=True,
                return_attention_mask = True,
              )

ورودی متناسب برای تنسرفلو باید این شکلی بشه

In [ ]:
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

گام ششم: درواقع میاد به ازای هر ورودی ساختار متناسب برای تسرفلوشو درمیاره و به هم میچسبونه تا تبدیل شه به یک دیتاست آماده برای تنسرفلو

In [ ]:
def encode_examples(data, labels, limit=-1):
  # prepare list, so that we can build up final TensorFlow dataset from slices.
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  label_list = []

  for review, label in zip(data, labels):
    bert_input = convert_example_to_feature(review)
    input_ids_list.append(bert_input['input_ids'])
    token_type_ids_list.append(bert_input['token_type_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
    label_list.append([label])
  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)

گام هفتم: پیش پردازش داده تموم شده و حالا نوبت دادن داده به شبکه هستش

In [ ]:
batch_size = 32
# train dataset
ds_train_encoded = encode_examples(train_text, train_labels).batch(batch_size)

# test dataset
ds_test_encoded = encode_examples(test_text, test_labels).batch(batch_size)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


خروجی بالا رو داره نشون  میده که چه اتفاقی داره میفته

In [ ]:
list(ds_train_encoded.as_numpy_iterator())

[({'input_ids': array([[    2,  4150,   623, ...,     0,     0,     0],
          [    2,  8029,  1923, ...,     0,     0,     0],
          [    2,  2746,  9365, ...,     0,     0,     0],
          ...,
          [    2, 26366,  2098, ...,     0,     0,     0],
          [    2,  2712,  2009, ...,     0,     0,     0],
          [    2,  6620,  2988, ...,     0,     0,     0]], dtype=int32),
   'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]], dtype=int32),
   'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
          [1, 1, 1, ..., 0, 0, 0],
          [1, 1, 1, ..., 0, 0, 0],
          ...,
          [1, 1, 1, ..., 0, 0, 0],
          [1, 1, 1, ..., 0, 0, 0],
          [1, 1, 1, ..., 0, 0, 0]], dtype=int32)},
  array([[[1., 0.]],
  
         [[0., 1.]],
  
         [[0., 1.]],
  
         [[0.,

گام هشتم: مدل پری ترین رو لود کنیم

zwnj  یعنی مدلمون نیم فاصله رو رعایت کرده

In [ ]:
from transformers import AutoTokenizer, TFBertForSequenceClassification

model = TFBertForSequenceClassification.from_pretrained("HooshvareLab/bert-fa-zwnj-base") #### چک شود

tf_model.h5:   0%|          | 0.00/603M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['classifier', 'bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


گام نهم: ساخت مدل فاین تیون

In [2]:
import tensorflow as tf

loss = tf.keras.losses.BinaryCrossentropy()
metric = tf.keras.metrics.BinaryAccuracy('accuracy')

model.compile(optimizer='adam', loss=loss, metrics=[metric])


NameError: name 'model' is not defined

In [ ]:
# import tensorflow as tf

# loss = tf.keras.losses.BinaryCrossentropy()
# metric = tf.keras.metrics.BinaryAccuracy('accuracy')

# model.compile(optimizer='adam',
#               learning_rate=2e-5,
#               epsilon=1e-08,
#               loss=loss,
#               metrics=[metric])


In [1]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08) #### تغییر عدد لرنینگ ریت
loss = tf.keras.losses.BinaryCrossentropy()
metric = tf.keras.metrics.BinaryAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])



NameError: name 'tf' is not defined

گام دهم: ترین کردن مدل

بهتره ایپاک 4 تا 5 کافی هست

In [ ]:
bert_history = model.fit(ds_train_encoded, epochs=2, validation_data=ds_test_encoded, ) ### تغییر ایپاک تا 10

Epoch 1/2


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
375/375 [==============================] - 417s 892ms/step - loss: 7.6095 - accuracy: 0.4950 - val_loss: 7.6246 - val_accuracy: 0.4989
Epoch 2/2
375/375 [==============================] - 323s 862ms/step - loss: 7.6246 - accuracy: 0.4992 - val_loss: 7.6246 - val_accuracy: 0.4995


پردیکت

In [ ]:
examples = ['به به چه غذای خوبی', 'خیلی عالی']
labels = one_hot([1, 1])
encoded = encode_examples(examples, labels).batch(batch_size)
encoded = list(encoded.as_numpy_iterator())[0][0]

preds_prob = model.predict({'input_ids':encoded['input_ids'], 'attention_mask':encoded['attention_mask']})
preds_prob.logits

1/1 [==============================] - 0s 88ms/step


array([[6.735039, 9.115447],
       [6.735039, 9.115447]], dtype=float32)

In [ ]:
np.argmax(preds_prob.logits, axis=1)

array([1, 1])